In [78]:
import os
# install requrements.txt
os.system('pip install -r requirements.txt')
%pip install -qU langchain-ollama
%pip install pypdf
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/80/4c/ee62b19a8daeed51e3c88c84b7da6047a74b786e598be3592b67a286d419/chromadb-0.5.5-py3-none-any.whl.metadata
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/e2/03/f3c8ba0a6b6e30d7d18c40faab90807c9bb5e9a1e3b2fe2008af624a9c97/build-1.2.1-py3-none-any.whl.metadata
  Using cached chroma_hnswlib-0.7.6-cp312-cp312-win_amd64.whl
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/5c/ae/6570ae1b67007735229f10f2e1174d6e33c056ee9c0c1754d432acbede94/fastapi-0.112.2-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pythonhosted.org/packages/f5/8e/cdc7d6263db313030e4c257dd5ba3909ebc4e4fb53ad62d5f09b1a2f5458/uvicorn-0.30.6-py3-none-any.whl.metadata
  Obtaining dependency information for posthog>=2.4.0 from https://files.pythonhosted.org/packages/94/e2/b71


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [74]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.globals import set_verbose
from langchain_core.messages import AIMessage
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma



load_dotenv()

True

## LLM Setup

In [35]:
gpt4 = ChatOpenAI(model="gpt-4")
parser = StrOutputParser()
json_parser = JsonOutputParser()

llama3 = ChatOllama(
    model="llama3.1:8b",
    temperature=0,
)


llama3_chain_text = llama3 | parser
gpt4_chain_text = gpt4 | parser

llama3_chain_json = llama3 | json_parser
gpt4_chain_json = gpt4 | json_parser

messages = [
    SystemMessage("translate to chinese"),
    HumanMessage("Hello, how are you?"),
]

message_generate_json = [
    SystemMessage("Give me a json of the translation from Eng to Chinese. 'Translation' is the outer key. Map should contain 'en' and 'cn' keys. Divided human message by newline\n."),
    HumanMessage("Hello, how are you? \n I am fine, thank you. \n"),
    HumanMessage("My name is Ze"),
]

messages2 = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Knock knock."),
    AIMessage(content="Who's there?"),
    HumanMessage(content="Orange"),
    
]

test_response = llama3_chain_text.invoke(messages2)

print(test_response)


Orange who?


## Basic knowledge feeding:

In [132]:

ZeSheng_info = [
    'Ze Sheng was born in Shenzhen, China in 2000. He came to the US in 2021 for his Master\'s degree at The University of Pennsylvania.',
    'In 2024, Ze graduated from the University of Pennsylvania with a Master\'s degree in Computer and Information Science.',
    'Currently, Ze is a phd student at Texas A&M University in the Department of Computer Science and Engineering, supervised by Dr.Jeff Huang.',
]

prompt_template = ChatPromptTemplate([
    ("system", "Answer the questions based on the following information about Ze Sheng: \n. If you don't know the answer, type 'I don't know'.\n"),
    ("system", ''.join(ZeSheng_info)),
    ("human", "{user_input}"),
])
msg_invoke = prompt_template.invoke("When was Ze born?")
test_response = llama3_chain_text.invoke(msg_invoke)
print(test_response)

msg_invoke = prompt_template.invoke("When did Ze come to the US?")
test_response = llama3_chain_text.invoke(msg_invoke)
print(test_response)

msg_invoke = prompt_template.invoke("Where is Ze now?")
test_response = llama3_chain_text.invoke(msg_invoke)
print(test_response)

msg_invoke = prompt_template.invoke("Who is Ze's father?")
test_response = llama3_chain_text.invoke(msg_invoke)
print(test_response)

1999.
2021.
Ze Sheng is currently at Texas A&M University, pursuing his PhD in the Department of Computer Science and Engineering.
I don't know.


In [163]:
pdf_loader = PyPDFLoader('attention_is_all_you_need.pdf')

pages = pdf_loader.load_and_split()

## Knowledge Sharding

### 1. Split Text into Chunks

In [164]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)

docs = text_splitter.split_documents(pages)

### 2. Embed Text Chunks and Store Them into Vector DB

In [167]:
embed_model = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=docs, embedding=embed_model , collection_name="zesheng3")

def augment_prompt(query: str):
    results = vectorstore.similarity_search(query, k=20)
    source_knowledge = "\n".join([x.page_content for x in results])

    prompt = ChatPromptTemplate([
        ("system", "Based on the following information, answer the question: \n"),
        ("system", source_knowledge),
        ("human", "{user_input}"),
    ])

    augment_prompt = prompt.invoke(query)

    return augment_prompt

query = "What English-to-French translation tasks are used in the paper?"

prompt = augment_prompt(query)

response = gpt4_chain_text.invoke(prompt)

print(response)

The paper used the WMT 2014 English-French dataset for the English-to-French translation tasks. This dataset consists of approximately 36 million sentences.


# Build A LLM with WEB search tool

In [75]:
tools = [TavilySearchResults(max_results=3)]
model = ChatOpenAI(model="gpt-4")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a great Assistant"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(model, tools, prompt)

set_verbose(True)


agent_executor = AgentExecutor(agent=agent, tools=tools)
response = agent_executor.invoke({"input": "What the release date of the movie 周处除三害?"})

print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '周处除三害 movie release date'}`


[{'url': 'https://www.imdb.com/title/tt29319990/releaseinfo/', 'content': 'Release Calendar Top 250 Movies Most Popular Movies Browse Movies by Genre Top Box Office Showtimes & Tickets ... Release Date. Taiwan. October 6, 2023; Hong Kong. November 2, 2023; Singapore. ... Os Três Males; Canada. The Pig, the Snake and the Pigeon (English) China. 周处除三害 (Mandarin) Germany. Zhou chu chu san hai; 9 more All ...'}, {'url': 'https://movie.douban.com/subject/36151692/', 'content': '周处除三害的剧情简介 · · · · · ·. 通缉犯陈桂林（阮经天 饰）在逃亡藏匿中失去最后一个亲人，同时得知自己只有不到三个月的生命。. 万念俱灰的他原打算投案自首，可是当发现他在三大通缉犯中仅仅排名第三时，内心突然躁动起来。. 在此之后，他决定 ...'}, {'url': 'https://www.themoviedb.org/movie/1046090?language=zh-CN', 'content': '通缉犯陈桂林（阮经天 饰）生命将尽，却发现自己在通缉榜上只排名第三，他决心查出前两名通缉犯的下落，并将他们一一除掉。陈桂林以为自己已成为当代的周处除三害，却没想到永远参不透的贪嗔痴，才是人生终要面对的罪与罚。 影片引用的"周处除三害"典故，见于《晋书·周处传》和 ...'}]The movie "周处除三害" is set to be released in Ta